# Projet maison n° 3

In [1]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [2]:
import os
# names us
def df_names_us():
    df = pd.DataFrame()
    filenames = os.listdir('./names/')
    filenames.sort()
    filenames.remove('NationalReadMe.pdf')
    df = pd.concat((pd.read_csv('./names/' + f, names=['name', 'gender', 'births']).assign(year=int(f[3:7])) 
                    for f in filenames))
    df.index = pd.RangeIndex(start=0, stop=len(df))
    return df[['year', 'name', 'gender', 'births']]

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [3]:
# names fr
def df_names_fr():
    df = pd.DataFrame()
    
    #Import CSV and change column names
    df = pd.read_csv('nat2021.csv', sep=';').drop(index = 0)
    df = df.rename(columns={'annais': 'year', 'sexe':'gender', 'preusuel':'name', 'nombre':'births'})
    
    # 1 is Male and 2 is Female
    df['gender'] = df['gender'].map({1: 'M', 2: 'F'})
    
    #Retirer les noms non conformes
    df = df[df['name'].apply(lambda x : len(str(x))) > 1]
    df = df[df['name'] != '_PRENOMS_RARES']
    
    #Replace '-'
    df['name'] = df['name'].str.replace('-','')
    
    #Trier
    df = df[df['year'] != 'XXXX']
    
    # Make names in normal format (first letter capital and rest lowercase)
    df['name'] = df['name'].astype(str).apply(lambda x: x.lower()).apply(lambda y: y.capitalize())
    
    #Sorting
    df = df.sort_values(by=['year', 'gender', 'births', 'name'], ascending=[True, True, False, True])
    
    df.index = pd.RangeIndex(start=0, stop=len(df))
    
    return df[['year', 'name', 'gender', 'births']]

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [4]:
def df_taux_change(devises):
    
    df = pd.read_csv("Webstat_Export.csv", sep=';')
    df = df.drop([0, 1, 2, 3, 4])
    df = df.rename(columns={"Titre :": "Date"})
    
    for column , _ in df.iteritems():
        if any(d in column for d in devises):
            df = df.rename(columns={column: column[-4:-1]})
        elif (column != "Date"):
            df = df.drop(columns=[column])
    
    df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")
    
    df = df.sort_values(by = ["Date"], ascending=[True])
    df = df.set_index("Date")
    
    for column , _ in df.iteritems():
        df = df[df[column] != '-']
    
    df = df.replace(',', '.', regex=True)
    df = df.astype('float')
    
    return df

In [5]:
df = pd.read_csv("Webstat_Export.csv",sep=';')
df.head()

,Titre :,Dollar australien (AUD),Lev bulgare (BGN),Real brésilien (BRL),Dollar canadien (CAD),Franc suisse (CHF),Yuan renminbi chinois (CNY),Livre chypriote (CYP),Couronne tchèque (CZK),Couronne danoise (DKK),...,Leu roumain (RON),Rouble russe (RUB),Couronne suédoise (SEK),Dollar de Singapour (SGD),Tolar slovène (SIT),Couronne slovaque (SKK),Baht thaïlandais (THB),Livre turque (TRY),Dollar des Etats-Unis (USD),Rand sud-africain (ZAR)
0,Code série :,EXR.D.AUD.EUR.SP00.A,EXR.D.BGN.EUR.SP00.A,EXR.D.BRL.EUR.SP00.A,EXR.D.CAD.EUR.SP00.A,EXR.D.CHF.EUR.SP00.A,EXR.D.CNY.EUR.SP00.A,EXR.D.CYP.EUR.SP00.A,EXR.D.CZK.EUR.SP00.A,EXR.D.DKK.EUR.SP00.A,...,EXR.D.RON.EUR.SP00.A,EXR.D.RUB.EUR.SP00.A,EXR.D.SEK.EUR.SP00.A,EXR.D.SGD.EUR.SP00.A,EXR.D.SIT.EUR.SP00.A,EXR.D.SKK.EUR.SP00.A,EXR.D.THB.EUR.SP00.A,EXR.D.TRY.EUR.SP00.A,EXR.D.USD.EUR.SP00.A,EXR.D.ZAR.EUR.SP00.A
1,Unité :,Dollar Australien (AUD),Lev Nouveau (BGN),Real Bresilien (BRL),Dollar Canadien (CAD),Franc Suisse (CHF),Yuan Ren Min Bi (CNY),Livre Cypriote (CYP),Couronne Tcheque (CZK),Couronne Danoise (DKK),...,Nouveau Ron (RON),Rouble Russe (RUB) (RUB),Couronne Suedoise (SEK),Dollar de Singapour (SGD),Tolar (SIT),Couronne Slovaque (SKK),Baht (THB),Nouvelle Livre Turque (TRY),Dollar des Etats-Unis (USD),Rand (ZAR)
2,Magnitude :,Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),...,Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0)
3,Méthode d'observation :,Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),...,Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A)
4,Source :,BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),...,BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0)


### Tests

In [6]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [7]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [8]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 4.272s

OK
